### 10.1 허프 변환

허프 변환 : 대표적인 직선 검출 방법, 데이터 손실 및 외곡이 발생한 영상에서 직선을 검출

##### 10.1.2 허프 변환의 전체 과정
1. 극좌표계에서 누적 행렬 구성
2. 영상 화소의 직선 검사
3. 직선 좌표에 대한 극좌표 누적 행렬 구성
4. 누적 행렬의 지역 최댓값 선정
5. 직선 선별 - 임계값 이상인 누적값 선택 및 정렬


In [1]:
# 직선 좌표들에 대해 극 좌표계로 변환하여 인덱스 구성
def accumulate(image, rho, theta):
    h, w = image.shape[:2]
    rows, cols = (h+w) * 2 // rho, int(np.pi / theta) # 누적행렬 너비, 높이
    accumulate = np.zeros((rows, cols), np.int32) # 직선 누적 행렬

    sin_cos = [(np.sin(t*theta), np.cos(t*theta)) for t in range(cols)]
    pts = np.where(image > 0)

    polars = np.dot(sin_cos, pts).T

    for row in polars:
        for t, r in enumerate(row):
            accumulate[r, t] += 1
    return accumulate



##### 10.1.4 누적 행렬의 지역 최댓값 선정
누적 행렬을 작은 블록으로 나눠, 각 블록에서 큰 값(지역 최대 값)만 보존하고 나머지를 제거


In [2]:
def masking(accumulate, h, w, thresh):
    rows, cols = accumulate.shape[:2]
    rcenter, tcenter = h//2, w//2
    dst = np.zeros(accumulate.shape, np.uint32)

    for y in range(0, rows, h):
        for x in range(0, cols, w):
            roi = accumulate[y:y+h, x:x+w]
            _, max, _, (x0, y0) = cv2.minMaxLoc(roi)
            dst[y+y0, x+x0] = max
    return dst

##### 10.1.5 직선(극 좌표) 선택 및 정렬
중복이 제거된 누적 행렬원소에서 임계값 보다 큰 값을 선별하여 별도로 저장

In [3]:
def select_lines(acc_dst, rho, theta, thresh):
    rows = acc_dst.shape[0]
    r, t = np.where(acc_dst > thresh)

    rhos = ((r - (rows / 2)) * rho)
    radians = t * theta
    values = acc_dst[r, t]

    idx = np.argsort(values)[::-1]
    lines = np.transpose([rhos, radians])

    return np.expand_dims(lines, axis=1)


In [1]:
# 10.1.1 허프 변환을 이용한 직선 검출
import numpy as np, cv2, math
from Common.hough import accumulate, masking, select_lines

def hough_lines(src, rho, theta, thresh):
    acc_mat = accumulate(src, rho, theta)
    acc_dst = masking(acc_mat, 7, 3, thresh)
    lines = select_lines(acc_dst, 7, 3, thresh)
    return lines

def draw_hough_lines(src, lines, nline):
    dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
    min_length = min(len(lines), nline)

    for i in range(min_length):
        rho, radian = lines[i, 0, 0:2]
        a, b = math.cos(radian), math.sin(radian)
        pt = (a * rho, b * rho)
        delta = (-1000 * b, 1000 * a)
        pt1 = np.add(pt, delta).astype('int')
        pt2 = np.subtract(pt, delta).astype('int')
        cv2.line(dst, tuple(pt1), tuple(pt2), (0, 255, 0), 2, cv2.LINE_AA)

    return dst

image = cv2.imread("images_10/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

blur = cv2.GaussianBlur(image, (5, 5), 2, 2)
canny = cv2.Canny(blur, 100, 200, 5)

rho, theta = 1, np.pi / 180
lines1 = hough_lines(canny, rho, theta, 80)
lines2 = cv2.HoughLines(canny, rho, theta, 80)
dst1 = draw_hough_lines(canny, lines1, 7)
dst2 = draw_hough_lines(canny, lines2, 7)

cv2.imshow('image', image)
cv2.imshow('canny', canny)
cv2.imshow('detected lines', dst1)
cv2.imshow('cv detected lines', dst2)
cv2.waitKey(0)


-1

In [1]:
# 10.1.2 직선 검출을 이용한 멀티 하네스 기울기 보정
import numpy as np, cv2
from Common.hough import *

def detect_maxObject(img):
    results = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if int(cv2.__version__[0]) >= 4:
        contours = results[0]
    else:
        contours = results[1]
    
    areas = [cv2.contourArea(c) for c in contours]
    idx = np.argsort(areas)
    max_rect = contours[idx[-1]]

    rect = cv2.boundingRect(max_rect)
    rect = np.add(rect, (-10, -10, 20, 20))
    return rect

image = cv2.imread("images_10/test.jpg", cv2.IMREAD_COLOR)
if image is None: raise Exception("영상 파일 읽기 오류")

rho, theta = 1, np.pi / 180
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
th_gray = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY)[1]
kernel = np.ones((3, 3), np.uint8)
morph = cv2.erode(th_gray, kernel, iterations=2)

x, y, w, h = detect_maxObject(np.copy(morph))
roi = th_gray[y:y+h, x:x+w]

canny = cv2.Canny(roi, 40, 100)
lines = cv2.HoughLines(canny, rho, theta, 50)

cv2.rectangle(morph, (x, y, w, h), 100, 2)
canny_line = draw_hough_lines(canny, lines, 1) ############################################ error
angle = (np.pi - lines[0, 0, 1]) * 180 / np.pi
h, w = image.shape[:2]
center = (w//2, h//2)
rot_map = cv2.getRotationMatrix2D(center, -angle, 1)
dst = cv2.warpAffine(image, rot_map, (w, h), cv2.INTER_LINEAR)

cv2.imshow("image", image)
cv2.imshow("morph", morph)
cv2.imshow("line_detect", canny_line)
cv2.imshow("line_detect", 250, canny_line.shape[0])
cv2.imshow("dst", dst)
cv2.waitKey(0)




TypeError: object of type 'NoneType' has no len()

In [6]:
# 10.3.1 임의 좌표 생성
import numpy as np, cv2

def draw_points(image, group, color):
    for p in group:
        pt = tuple(p.astype(int))
        cv2.circle(image, pt, 3, color, cv2.FILLED)

nsample = 50
traindata = np.zeros((nsample*2, 2), np.float32)
label = np.zeros((nsample*2, 1), np.float32)

cv2.randn(traindata[:nsample], 150, 30)
cv2.randn(traindata[:nsample], 250, 60)
label[:nsample], label[nsample:] = 0, 1

K = 7
knn = cv2.ml.KNearest_create()
knn.train(traindata, cv2.ml.ROW_SAMPLE, callable) # errrrrrrrrrrrrrrrrrrrrrrrror

points = [(x, y) for y in range(400) for x in range(400)]
ret, resp, neig, dist = knn.findNearest(np.array(points, np.float32), K)

colors = [(0, 180, 0) if p else (0, 0, 180) for p in resp]
image = np.reshape(colors, (400, 400, 3)).astype('uint8')

draw_points(image, traindata[:nsample], color=(0, 0, 255))
draw_points(image, traindata[nsample:], color=(0, 255, 0))
cv2.imshow("sample K =" + str(K), image)
cv2.waitKey(0)

error: OpenCV(4.5.4-dev) :-1: error: (-5:Bad argument) in function 'train'
> Overload resolution failed:
>  - ml_StatModel.train() takes at most 2 arguments (3 given)
>  - responses is not a numpy array, neither a scalar
>  - Expected Ptr<cv::UMat> for argument 'responses'
